# Cross Attention

##### cross Attention  
与selfAttention非常类似，但指的是一个序列和另一个序列的相关性
> 一个序列提供Q，另外一个序列提供K、V，存在个别一个序列提供K，另一个序列提供Q、V的情况
> transformer中Decode的第二个MultiHeadAttention就用的cross Attention

In [3]:
import math
import torch
import torch.nn as nn

# MultiHeadAttention是一个特征提取器
# 输入query, key, value三个向量，输出部分是融合了上下文语义信息的单词表示，输出维度和query相同
# 可以兼容transformer中的三类Attention：encoder self-attention，无mask，输入query = key = value
# decoder self-attention，有sequence mask，保证当前单词只能看到之前的单词，看不到之后的单词。输入query = key = value
# encoder-decoder attention，实现encoder和decoder的交互，query是decoder层的输入，key = value 为encoder的输出。


class MultiHeadAttention(nn.Module):
    def __init__(self, heads, hidden_size):
        super(MultiHeadAttention, self).__init__()
        assert hidden_size % heads == 0
        self.hidden_size = hidden_size
        self.heads = heads
        self.wq = nn.Linear(hidden_size, hidden_size)
        self.wk = nn.Linear(hidden_size, hidden_size)
        self.wv = nn.Linear(hidden_size, hidden_size)

    def forward(self, query, key, value, mask=None):
        # query, key, value = [batch_size, seq_len, hidden_size]
        batch_size, seq2_len, hidden_size = query.shape
        seq1_len = key.shape[1]
        q = self.wq(query).view(
            batch_size, seq2_len, self.heads, -1
        )  # [batch_size, seq2_len, heads, d_k]
        k = self.wk(key).view(
            batch_size, seq1_len, self.heads, -1
        )  # [batch_size, seq1_len, heads, d_k]
        v = self.wv(value).view(
            batch_size, seq1_len, self.heads, -1
        )  # [batch_size, seq1_len, heads, d_k]
        q = q.permute(0, 2, 1, 3)  # [batch_size, heads, seq2_len, d_k]
        k = k.permute(0, 2, 1, 3)  # [batch_size, heads, seq1_len, d_k]
        v = v.permute(0, 2, 1, 3)  # [batch_size, heads, seq1_len, d_k]
        d_k = self.hidden_size // self.heads
        attention = torch.matmul(q, k.transpose(2, 3)) / math.sqrt(
            d_k
        )  # [batch_size, heads, seq2_len, seq1_len]

        if mask is not None:
            if len(mask.shape) != len(attention.shape):
                mask.unsqueeze_(1)  # [batch_size, 1, seq2_len, seq1_len]
            attention.masked_fill_(mask, float("-inf"))
        score = nn.functional.softmax(attention, dim=-1)
        output = torch.matmul(score, v)  # [batch_size, heads, seq2_len, d_k]
        output = output.permute(0, 2, 1, 3).reshape(
            batch_size, seq2_len, -1
        )  # [batch_size, seq2_len, heads, d_k] -> [batch_size, seq2_len, hidden_size]
        return output


class FeedForward(nn.Module):
    def __init__(self, model_size, dropout=0.1):
        super(FeedForward, self).__init__()
        self.linear1 = nn.Linear(model_size, 4 * model_size)
        self.linear2 = nn.Linear(4 * model_size, model_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.relu(self.linear1(x))
        return self.linear2(self.dropout(x))

测试代码

In [5]:
def testCrossAttention():
    x = torch.randn(2, 3, 4)
    y = torch.randn(2, 5, 4)

运行代码

In [6]:
if __name__ == "__main__":
    testCrossAttention()

tensor([[[ True, False,  True],
         [ True, False,  True],
         [ True, False,  True],
         [ True, False,  True]],

        [[ True, False,  True],
         [ True, False,  True],
         [ True, False,  True],
         [ True, False,  True]]])
tensor([[[-0.2809,  0.2711,  0.2169, -0.2530,  0.2813,  0.8875],
         [-1.3327, -0.6412, -0.4290,  0.1109, -2.0021, -0.7308],
         [-1.9483, -1.2553,  0.1657,  0.6736, -1.2003, -2.5043],
         [ 0.9923,  1.4159, -1.4476,  1.0222, -0.0945,  1.5952]],

        [[ 0.9543, -0.7563,  2.4448, -0.4666, -0.2174,  1.4866],
         [ 0.5999,  1.0395,  0.5767, -1.2021, -1.4562, -0.4506],
         [-0.6769, -0.9013,  1.5433,  0.6820, -0.6421,  0.7189],
         [-1.4438,  1.3823, -0.0441, -0.6321,  0.1824, -0.5873]]])
tensor([[[ 1.3434, -0.9425, -1.0707,  0.1986, -1.9386, -0.5820],
         [ 0.6702,  1.3535,  1.6045, -0.0666, -1.0182, -0.2781],
         [ 0.3260, -1.0867, -1.1096,  1.9052, -0.3778, -0.3834]],

        [[-1.9636, 